### 랭체인 주요기능 
- 문맥 인식: 언어 모델과 다양한 문맥 소스(프롬프트 지시, 예제, 응답의 근거 내용 등)를 연동하며, 사용자의 문맥을 정확히 이해합니다.
- 추론 능력: 제공된 문맥에 기반하여 어떤 대답을 할지, 또는 어떠한 액션을 취할지에 대한 추론이 가능합니다.
- 구성 요소: 사용자는 언어 모델과의 상호작용을 위해 다양한 구성 요소와 추상화를 활용할 수 있습니다. 이러한 구성 요소는 개별적으로, 또는 랭체인 프레임워크 내에서 모듈식으로 쉽게 활용할 수 있습니다.
- 사용 준비된 체인: 특정 고수준 작업을 수행하기 위해 미리 조립된 구성 요소의 패키지입니다.

In [ ]:
pip install openai langchain dotenv langchain_community

In [ ]:
from dotenv import load_dotenv
import os
import openai

load_dotenv('../.env')  # .env 파일의 환경변수 로드
api_token = os.environ.get("OPENAI_API_KEY")

# openai 버전이 1.x.x 이하일경우 
"""
openai.api_key = api_token

models = openai.Model.list()
for model in models['data']:
    print(model.id)
"""

# openai 버전이 1.x.x 이상일경우 
client = openai.OpenAI(api_key=api_token)

models = client.models.list()
for model in models.data:
    print(model.id)

In [10]:
from langchain.chat_models import ChatOpenAI

gpt41 = ChatOpenAI(model="gpt-4.1-nano", temperature=0, max_tokens=100)
question = "대한민국의 수도는 뭐야?"
print(f'{question} 에 대한 답변 : {gpt41.predict(question)}')

대한민국의 수도는 뭐야? 에 대한 답변 : 대한민국의 수도는 서울입니다.


* PromptTemplate  
사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다  
사용법 :  
template: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 {}는 변수를 나타냅니다.  
input_variables: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.  

* input_variables  
input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.  
사용법: 리스트 형식으로 변수 이름을 정의합니다.

### LLMChain 객체
LLMChain은 특정 PromptTemplate와 연결된 체인 객체를 생성합니다  

사용법  
- prompt: 앞서 정의한 PromptTemplate 객체를 사용합니다.
- llm: 언어 모델을 나타내며, 이 예시에서는 이미 어딘가에서 정의된 것으로 보입니다.
- run() 함수로 템플릿 프롬프트 실행
- apply() 함수로 여러개의 입력을 한 번에 실행
- generate() 는 문자열 대신에 LLMResult를 반환하는 점을 제외하고는 apply와 유사

In [11]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

temp_question = '{country}의 수도는 뭐야?'
country_prompt = PromptTemplate(template=temp_question, input_variables=['country'])
gpt41_chain = LLMChain(llm=gpt41, prompt=country_prompt)
print(gpt41_chain.run(country='대한민국'))
print(gpt41_chain.run(country='미국'))

대한민국의 수도는 서울입니다.
미국의 수도는 워싱턴 D.C.입니다.


In [14]:
input_list = [
    {'country': '호주'},
    {'country': '중국'},
    {'country': '네덜란드'}
]

output_list =gpt41_chain.apply(input_list)
for i in range(len(output_list)):
    print(f'{output_list[i]["text"]}')

호주의 수도는 캔버라입니다.
중국의 수도는 베이징입니다.
네덜란드의 수도는 암스테르담입니다.


In [18]:
generate_output = gpt41_chain.generate(input_list)
#generate_output.generations
#generate_output.llm_output

# 답변 출력
for gen in generate_output.generations:
    print(gen[0].text.strip())

호주의 수도는 캔버라입니다.
중국의 수도는 베이징입니다.
네덜란드의 수도는 암스테르담입니다.
